In [7]:
import torch
import numpy as np
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import timm
import detectors
from src.temperature_scaling import ModelWithTemperature
from src.aps import split_data_set, aps_scores, aps_classification, eval_aps

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet50_cifar100", pretrained=True)
model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../data", train=False, download=True, transform=data_transform)

# Tempreture Scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=4.85).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1

# contruct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = aps_scores(model, calib_loader, alpha, device)
    q_hat = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"q_hat = {q_hat}")

    # construct APS
    aps, aps_labels, true_labels = aps_classification(model, test_loader, q_hat, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Ratein runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)

print(f"Final Average Prediction Set Size: {final_avg_set_size}")
print(f"Final Average Coverage: {final_avg_coverage}")

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti
Files already downloaded and verified
Before temperature - NLL: 1.067, ECE: 0.086
Optimal temperature: 4.691
After temperature - NLL: 2.948, ECE: 0.673
APS Classification, Start!

Running experiment 1/10...
q_hat = 0.15436151939298803
Total set size: 19435
Total coverage sets: 4541
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 3.887
Average Coverage Ratein runs 1: 0.9082

Running experiment 2/10...
q_hat = 0.15162377986564843
Total set size: 18527
Total coverage sets: 4507
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 3.7054
Average Coverage Ratein runs 2: 0.9014

Running experiment 3/10...
q_hat = 0.1484622369477019
Total set size: 17929
Total coverage sets: 4475
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 3.5858
Average Coverage Ratein runs 3: 0.895

Running experiment 4/10...
q_hat = 0.1492952031961142
Total set s

# Result

- Final Average **Prediction Set Size： 3.66 / 100**
- Final Average **Coverage: 89.83% ($\alpha$=0.1)**